# Wafer defect classification in silicon manufacturing

Data from kaggle dataset

Overall goals:
* Explore dataset to get a feel for what's in it and any quick wins
    * Do some drill-down via pivot tables and exploratory graphs
* Perform supervised machine learning to be able to predict if a wafer will be defective or not defective
* Perform metaheuristic optimization in order to suggest optimal operating parameters

In [14]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

data_path = '../../data/uci-secom.csv'

In [18]:
# this comes from fast.ai
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 100): 
        display(df)

### Exploratory Data Analysis

In [19]:
# read in data to work with
df = pd.read_csv(data_path)
display_all(df.describe().T)

,count,mean,std,min,25%,50%,75%,max
0,1561.0,3014.452896,73.621787,2743.2400,2966.260000,3011.49000,3056.650000,3356.3500
1,1560.0,2495.850231,80.407705,2158.7500,2452.247500,2499.40500,2538.822500,2846.4400
2,1553.0,2200.547318,29.513152,2060.6600,2181.044400,2201.06670,2218.055500,2315.2667
3,1553.0,1396.376627,441.691640,0.0000,1081.875800,1285.21440,1591.223500,3715.0417
4,1553.0,4.197013,56.355540,0.6815,1.017700,1.31680,1.525700,1114.5366
5,1553.0,100.000000,0.000000,100.0000,100.000000,100.00000,100.000000,100.0000
6,1553.0,101.112908,6.237214,82.1311,97.920000,101.51220,104.586700,129.2522
7,1558.0,0.121822,0.008961,0.0000,0.121100,0.12240,0.123800,0.1286
8,1565.0,1.462862,0.073897,1.1910,1.411200,1.46160,1.516900,1.6564
9,1565.0,-0.000841,0.015116,-0.0534,-0.010800,-0.00130,0.008400,0.0749


In [23]:
df.shape

(1567, 592)

In [26]:
df.columns

Index(['Time', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '581', '582', '583', '584', '585', '586', '587', '588', '589',
       'Pass/Fail'],
      dtype='object', length=592)

In [27]:
df.Time.min()

'2008-01-08 02:02:00'

In [28]:
df.Time.max()

'2008-12-10 18:47:00'

Some initial thoughts just looking at this:
* We have some data that has zero variance- we will want to get rid of this
* We should look for highly linearly correlated features
* Most of the data looks numerical rather than categorical
* Columns names tell us nothing- maybe we re-label these to not confuse ourselves
* We should look for index-based trends as these likely are related to time dependencies of our data
* Data appears to be in completely different scales- any non tree-based method will need scaling done to it
* Should probably look for outliers as well since there are some data that appear to be skewed on first glance
* We onlt have ~3x as many observations as predictor fields...
    * More urgent need for dimensionality reduction
    * Can be done through PCA or other means
* Date ranges over the course of the year
    * Maybe look at typical scheduling routines
        * Time based features for 8 hour or 12 hour shifts
    * Likely won't have enough data to represent full seasonality over the course of multiple years